# PoliGPT - Conversação RAG

In [1]:
import os

os.environ["AZURESEARCH_FIELDS_ID"] = "chunk_id"
os.environ["AZURESEARCH_FIELDS_CONTENT"] = "chunk"
os.environ["AZURESEARCH_FIELDS_CONTENT_VECTOR"] = "vector"

In [2]:
from langchain_core.tools import tool
from langchain_core.messages import trim_messages
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_community.vectorstores.azuresearch import AzureSearch, AzureSearchVectorStoreRetriever

In [3]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

## Parameters

In [4]:
# Conectar ao Azure Key Vault
kv_uri = "https://kv-poligpt-dev-eastus2.vault.azure.net"
credential = DefaultAzureCredential()
client = SecretClient(vault_url=kv_uri, credential=credential)

In [5]:
OPENAI_API_KEY = client.get_secret("openai-api-key").value
COMPLETION_MODEL = "gpt-4o-2024-08-06"
EMBEDDING_MODEL = "text-embedding-3-large"
EMBEDDING_DIMENSIONS = 3072
MODEL_TEMPERATURE = 0.3

AZURE_SEARCH_ENDPOINT = client.get_secret("azure-search-endpoint").value
AZURE_SEARCH_ADMIN_KEY = client.get_secret("azure-search-admin-key").value
SEARCH_INDEX_NAME = "poligpt-index"
SEARCH_TYPE = "hybrid"
NUM_DOCS_TO_RETRIEVE = 5

## Setup

### LLM

In [6]:
llm = ChatOpenAI(
    model=COMPLETION_MODEL,
    api_key=OPENAI_API_KEY,
    streaming=True,
    temperature=0.3
)

### Embeddings

In [7]:
embeddings = OpenAIEmbeddings(
    model=EMBEDDING_MODEL,
    openai_api_key=OPENAI_API_KEY, 
    dimensions=EMBEDDING_DIMENSIONS
)

### Vector Store

In [8]:
vector_store = AzureSearch(
    azure_search_endpoint=AZURE_SEARCH_ENDPOINT,
    azure_search_key=AZURE_SEARCH_ADMIN_KEY,
    index_name=SEARCH_INDEX_NAME,
    embedding_function=embeddings.embed_query,
    search_type="hybrid"
)

In [49]:
# # Teste do vector store
# docs = vector_store.hybrid_search(
#     query="Como me inscrevo numa materia?",
#     k=3
# )

# docs

In [50]:
# retriever = AzureSearchVectorStoreRetriever(
#     name="AzureSearchRetriever",
#     vectorstore=vector_store,
#     search_type="hybrid",
#     k=3
# )

## RAG Agent

In [16]:
memory = MemorySaver()

In [10]:
@tool(response_format="content_and_artifact")
def retriever_tool(query: str):
    """Busca e retorna informações relacionadas a questões acadêmicas ou institucionais sobre a Escola Politécnica da UFRJ."""
    retrieved_docs = vector_store.hybrid_search(query, k=3)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [11]:
tools = [retriever_tool]

In [17]:
agent_executor = create_react_agent(
    model=llm, 
    tools=tools, 
    checkpointer=memory
)

In [30]:
config = {"configurable": {"thread_id": "def234"}}

input_message = (
    "A politecnica ja recebeu algum prêmio?"
)

events = []
for event in agent_executor.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    event["messages"][-1].pretty_print()
    #events.append(event)

================================ Human Message =================================

A politecnica ja recebeu algum prêmio?
================================== Ai Message ==================================

Sim, a Escola Politécnica da UFRJ já recebeu prêmios. Recentemente, ela foi contemplada com sete premiações no XI Prêmio Crea-RJ de Trabalhos Científicos e Tecnológicos. Esses prêmios foram concedidos a trabalhos de conclusão de curso, incluindo um do mestrado do Programa de Engenharia Urbana e seis de cursos de graduação nas engenharias Ambiental, Civil, Metalúrgica, Materiais, Petróleo e Produção. Além disso, alunos da Politécnica-UFRJ também receberam prêmios da ABS, e uma ONG criada por uma ex-aluna foi premiada pela Women of the Future Programme (WOF) e pela Unesco.


: 

In [26]:
events[-1]["messages"][-1].content

'Sim, a Escola Politécnica da UFRJ já recebeu prêmios. Recentemente, no XI Prêmio Crea-RJ de Trabalhos Científicos e Tecnológicos, a Escola Politécnica foi contemplada com sete premiações para trabalhos de conclusão de curso, incluindo um do mestrado do Programa de Engenharia Urbana e seis de cursos de graduação nas engenharias Ambiental, Civil, Metalúrgica, Materiais, Petróleo e Produção. Além disso, alunos da Politécnica-UFRJ também foram premiados pela ABS, e uma ONG criada por uma ex-aluna foi premiada pela Women of the Future Programme (WOF) e pela Unesco.'

In [29]:
events

[{'messages': [HumanMessage(content='A politecnica ja recebeu algum prêmio?', additional_kwargs={}, response_metadata={}, id='8af61aae-3166-4ad4-b82e-7ce78d923efe'),
   AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_QH3fKf5cLFRIlkj7UQfDmeTA', 'function': {'arguments': '{"query":"prêmios recebidos pela Escola Politécnica da UFRJ"}', 'name': 'retriever_tool'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87'}, id='run-b93770ba-61db-47b7-81d0-22dbbd80e58c-0', tool_calls=[{'name': 'retriever_tool', 'args': {'query': 'prêmios recebidos pela Escola Politécnica da UFRJ'}, 'id': 'call_QH3fKf5cLFRIlkj7UQfDmeTA', 'type': 'tool_call'}]),
   ToolMessage(content="Source: {'chunk_id': '52fc867f789a_7fabf1011cee4c4219f6290f4b9592bf_pages_0', 'keywords': 'noticia,crea rj premia sete trabalhos cientificos da escola politecnica', 'parent_id': '7fabf1011cee4c4219f6290f4b9592b

---

## Testes com Agente 

In [56]:
retriever = AzureSearchVectorStoreRetriever(
    name="AzureSearchRetriever",
    vectorstore=vector_store,
    search_type="hybrid",
    k=3
)

In [57]:
memory = MemorySaver()

In [58]:
retriever_tool = create_retriever_tool(
    retriever = retriever,
    name = "buscar_poli_info",
    description = "Busca e retorna informações relacionadas a questões acadêmicas ou institucionais sobre a Escola Politécnica da UFRJ.",
)

tools = [retriever_tool]

In [59]:
system_message = SystemMessage(
    content="""
    Você é o PoliGPT, um assistente virtual da Escola Politécnica da Universidade Federal do Rio de Janeiro (UFRJ), que auxilia alunos, professores, funcionários e outros interessados 
    em questões acadêmicas e institucionais relacionadas à Escola Politécnica, além de conhecimentos gerais. Quaisquer perguntas relacionadas à área acadêmica feitas pelo
    usuário devem ser por padrão consideradas como referentes à Escola Politécnica, a não ser que seja explicitamente dito o contrário. Caso não tenha informações suficientes 
    para responder à pergunta, diga que não sabe. Responda sempre na mesma língua usada pelo usuário; caso não seja possível reconhecer a língua, use português.
    """
)

agent_executor = create_react_agent(
    model=llm, 
    tools=tools, 
    state_modifier=system_message,
    checkpointer=memory
)
agent_executor.step_timeout = 10

In [60]:
query = "Meu nome é Barbara."

config = {"configurable": {"thread_id": "test-thread"}}
messages = agent_executor.invoke(input={"messages": [("user", query)]}, config=config)

In [61]:
print(messages["messages"][-1].content)

Olá, Barbara! Como posso ajudar você hoje?


---

In [62]:
chat_history = [
    system_message,
    HumanMessage(content="Oi! Meu nome é Julia."),
    AIMessage(content="Ola!"),
    HumanMessage(content="Como voce esta?"),
    AIMessage(content="Bem, obrigado!"),
]

trimmed_history = trim_messages(
    chat_history,
    strategy="last",
    token_counter=llm,
    max_tokens=180,
    start_on="human",
    end_on="ai",
    include_system=True,
    allow_partial=False
)

In [63]:
query = "Tem noticias relevantes recentes sobre a escola politecnica?"

chat_messages = trimmed_history + [HumanMessage(content=query)]

config = {"configurable": {"thread_id": "test-thread-2"}}

messages = agent_executor.invoke(
    input={"messages": chat_messages}, 
    config=config
)

print(messages["messages"][-1].content)

KeyboardInterrupt: 

In [ ]:
messages

{'messages': [SystemMessage(content='\n    Você é o PoliGPT, um assistente virtual da Escola Politécnica da Universidade Federal do Rio de Janeiro (UFRJ), que auxilia alunos, professores, funcionários e outros interessados \n    em questões acadêmicas e institucionais relacionadas à Escola Politécnica, além de conhecimentos gerais. Quaisquer perguntas relacionadas à área acadêmica feitas pelo\n    usuário devem ser por padrão consideradas como referentes à Escola Politécnica, a não ser que seja explicitamente dito o contrário. Caso não tenha informações suficientes \n    para responder à pergunta, diga que não sabe. Responda sempre na mesma língua usada pelo usuário; caso não seja possível reconhecer a língua, use português.\n    ', additional_kwargs={}, response_metadata={}, id='fe17a7db-b9b0-4b03-8430-8b3dcac01dfe'),
  HumanMessage(content='Oi! Meu nome é Julia.', additional_kwargs={}, response_metadata={}, id='04862b72-363e-416d-a7b4-66df9576862a'),
  AIMessage(content='Ola!', addit

In [ ]:
agent_executor.get_state(config=config)

StateSnapshot(values={'messages': [SystemMessage(content='\n    Você é o PoliGPT, um assistente virtual da Escola Politécnica da Universidade Federal do Rio de Janeiro (UFRJ), que auxilia alunos, professores, funcionários e outros interessados \n    em questões acadêmicas e institucionais relacionadas à Escola Politécnica, além de conhecimentos gerais. Quaisquer perguntas relacionadas à área acadêmica feitas pelo\n    usuário devem ser por padrão consideradas como referentes à Escola Politécnica, a não ser que seja explicitamente dito o contrário. Caso não tenha informações suficientes \n    para responder à pergunta, diga que não sabe. Responda sempre na mesma língua usada pelo usuário; caso não seja possível reconhecer a língua, use português.\n    ', additional_kwargs={}, response_metadata={}, id='fe17a7db-b9b0-4b03-8430-8b3dcac01dfe'), HumanMessage(content='Oi! Meu nome é Julia.', additional_kwargs={}, response_metadata={}, id='04862b72-363e-416d-a7b4-66df9576862a'), AIMessage(cont

---

In [ ]:
import asyncio

In [68]:
async def setup_llm(
    temperature: float = 0.3
) -> ChatOpenAI:

    llm = ChatOpenAI(
        model=COMPLETION_MODEL,
        api_key=OPENAI_API_KEY,
        streaming=False,
        temperature=temperature
    )

    return llm

async def setup_retriever(
    search_type: str = "hybrid", 
    k: int = 3
) -> AzureSearchVectorStoreRetriever:

    embeddings = OpenAIEmbeddings(
        model=EMBEDDING_MODEL,
        openai_api_key=OPENAI_API_KEY, 
        dimensions=EMBEDDING_DIMENSIONS
    )

    vector_store = AzureSearch(
        azure_search_endpoint=AZURE_SEARCH_ENDPOINT,
        azure_search_key=AZURE_SEARCH_ADMIN_KEY,
        index_name=SEARCH_INDEX_NAME,
        embedding_function=embeddings.embed_query,
        search_type=search_type
    )

    retriever = AzureSearchVectorStoreRetriever(
        name="AzureSearchRetriever",
        vectorstore=vector_store,
        search_type=search_type,
        k=k
    )

    return retriever


async def agent_orchestrator(
    query: str, 
    chat_history: list,
    session_id: str
) -> str:

    memory = MemorySaver()
    llm = setup_llm(temperature=MODEL_TEMPERATURE)
    retriever = setup_retriever(search_type=SEARCH_TYPE, k=NUM_DOCS_TO_RETRIEVE)

    # Definir tool que irá realizar a busca vetorial
    retriever_tool = create_retriever_tool(
        retriever = retriever,
        name = "buscar_poli_info",
        description = "Busca e retorna informações relacionadas a questões acadêmicas ou institucionais sobre a Escola Politécnica da UFRJ.",
    )

    tools = [retriever_tool]

    # Definir mensagem de sistema para direcionar as respostas
    system_message = SystemMessage(
        content="""
        Você é o PoliGPT, um assistente virtual da Escola Politécnica da Universidade Federal do Rio de Janeiro (UFRJ), que auxilia alunos, professores, funcionários e outros interessados 
        em questões acadêmicas e institucionais relacionadas à Escola Politécnica, além de conhecimentos gerais. Quaisquer perguntas relacionadas à área acadêmica feitas pelo
        usuário devem ser por padrão consideradas como referentes à Escola Politécnica, a não ser que seja explicitamente dito o contrário. Caso não tenha informações suficientes 
        para responder à pergunta, diga que não sabe. Responda sempre na mesma língua usada pelo usuário; caso não seja possível reconhecer a língua, use português.
        """
    )

    # Definir agente executor do RAG com acesso às tools propostas
    agent_executor = create_react_agent(
        model=llm, 
        tools=tools, 
        state_modifier=system_message,
        checkpointer=memory
    )

    # Transformar o histórico de mensagens para o formato esperado pelo agente
    processed_history = [system_message]  # A mensagem do sistema deve ser a primeira da lista
    for message in chat_history:
        content = message["content"]
        if message["role"] == "user":
            processed_history += [HumanMessage(content=content)]
        elif message["role"] == "ai":
            processed_history += [AIMessage(content=content)]
        else:
            raise ValueError("O histórico de mensagens contém mensagens inválidas.")
        
    # Adicionar o prompt atual ao histórico
    processed_history += [HumanMessage(content=query)]

    # Remover mensagens antigas do histórico de mensagens
    trimmed_history = trim_messages(
        processed_history,
        strategy="last",
        token_counter=llm,
        max_tokens=8192,  # máximo de tokens que serão "lembrados" pelo modelo
        start_on="human",
        end_on="human",
        include_system=True,
        allow_partial=False
    )

    # Invocar o executor do agente para responder à pergunta do usuário
    config = {"configurable": {"thread_id": session_id}}
    messages = await agent_executor.ainvoke(input={"messages": trimmed_history}, config=config)

    # Extrair resposta final
    output = messages["messages"][-1].content

    return output

In [69]:
session_id = "001"

chat_history = [
    {"role": "user", "content": "Oi! Meu nome é Julia."},
    {"role": "ai", "content": "Ola!"},
    {"role": "user", "content": "Como voce esta?"},
    {"role": "ai", "content": "Bem, obrigado!"},
]

In [70]:
# %%async

query = "Qual foi minha ultima mensagem?"

answer = await agent_orchestrator(query, chat_history, session_id)
print(answer)

AttributeError: 'coroutine' object has no attribute 'bind_tools'

In [ ]:
# Atualizar histórico
last_user_message = {"role": "user", "content": query}
last_ai_message = {"role": "ai", "content": answer}
chat_history += [last_user_message] + [last_ai_message]

In [36]:
chat_history

[{'role': 'user', 'content': 'Oi! Meu nome é Julia.'},
 {'role': 'ai', 'content': 'Ola!'},
 {'role': 'user', 'content': 'Como voce esta?'},
 {'role': 'ai', 'content': 'Bem, obrigado!'},
 {'role': 'user', 'content': 'Qual o meu nome?'},
 {'role': 'ai', 'content': 'Seu nome é Julia. Como posso ajudar você hoje?'},
 {'role': 'user', 'content': 'Qual foi minha ultima pergunta?'},
 {'role': 'ai', 'content': 'Sua última pergunta foi: "Como voce esta?"'},
 {'role': 'user', 'content': 'Qual foi minha ultima mensagem, eu quis dizer?'},
 {'role': 'ai', 'content': 'Sua última mensagem foi: "Qual o meu nome?"'},
 {'role': 'user',
  'content': 'Qual o procedimento para ingressar na escola politecnica?'},
 {'role': 'ai',
  'content': 'Para ingressar na Escola Politécnica da UFRJ, geralmente é necessário participar do processo seletivo do Sistema de Seleção Unificada (SiSU), que utiliza as notas do Exame Nacional do Ensino Médio (ENEM). Após ser aprovado, os novos alunos são recepcionados durante a S